
# INFO-H600 : Million Playlist Dataset (Spotify) -Q.4

## Stratégie et Méthodes

1.  **Gestion des Données (Tâche 1) :** Le dataset de $35$ GB sera géré à l'aide de frameworks distribués (Dask/Spark) pour la scalabilité[cite: 9, 18].
2.  **Analyse Thématique (Tâche 4) :**
    * **Vectorisation :** Nous appliquons le **Latent Dirichlet Allocation (LDA)** aux métadonnées textuelles (titres de playlists/pistes/artistes) pour extraire des **sujets thématiques** clés.
    * **Similarité Playlist :** La similarité entre deux playlists est définie par la distance (Similarité Cosinus) entre leurs vecteurs de probabilités thématiques.




# Step 0 – Aller sur kaggle créer un compte et reccup le fichier avec la clé
# Step 0.1 – Configurer l'API Kaggle dans Colab

Objectif :
- Utiliser le fichier `kaggle.json` (qui contient que la clé API) déjà uploadé dans Colab.
- Le copier dans le dossier spécial `~/.kaggle/`.
- Vérifier que l'API Kaggle fonctionne.

Précondition :
- Le fichier `kaggle.json` doit se trouver dans `/content/kaggle.json`
  (tu l'as déjà uploadé via l'interface fichiers de Colab).


In [1]:
import os
import shutil

# Installer/mettre à jour l'API Kaggle
!pip install -q kaggle --upgrade

# Créer le dossier .kaggle si besoin
os.makedirs("/root/.kaggle", exist_ok=True)

# Copier le kaggle.json que tu as uploadé dans /content
src = "/content/kaggle.json"
dst = "/root/.kaggle/kaggle.json"

shutil.copy(src, dst)
os.chmod(dst, 0o600)

print("✔️ kaggle.json copié et permissions configurées")

# Vérifier que l'API répond
!kaggle -v


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 17.9 MB/s eta 0:00:00
✔️ kaggle.json copié et permissions configurées
Kaggle API 1.8.2


# Step 1 – Vérifier l'accès au dataset Spotify et préparer le dossier de travail

Objectif :
- Vérifier que l'API Kaggle peut accéder au dataset "spotify-million".
- Créer un dossier local `/content/mpd` pour stocker les fichiers JSON.
- Ne **pas** télécharger tout le dataset, juste quelques fichiers ciblés ensuite.


In [2]:
# Vérifier que le dataset est accessible
!kaggle datasets list -s "spotify-million"

# Créer le dossier de travail pour les slices qu'on va reccup après
!mkdir -p /content/mpd

print("✔️ Dossier /content/mpd prêt")


ref                                                      title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  -----------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
notshrirang/spotify-million-song-dataset                 Spotify Million Song Dataset                       21735868  2022-11-21 16:48:45.780000          21833        143                1  
undefinenull/million-song-dataset-spotify-lastfm         Million Song Dataset + Spotify + Last.fm          670020079  2023-04-22 08:17:45.377000           7060         71        0.9411765  
barirasajjad/spotify-dataset                             Spotify_dataset                                    22073301  2024-06-04 00:53:18.953000             39         16        0.4117647  
adnananam/spotify-artist-stats                    




# Step 2 V2 – Inspecter le contenu du dataset Kaggle

(**UPDATE** : on ne peut pas faire -f mpd.slice.0-999.json → 404
il faut d’abord télécharger un ou quelques gros fichiers, puis extraire seulement ce qu’on veut)

Objectif :
- Lister tous les fichiers disponibles dans le dataset `himanshuwagh/spotify-million`.
- Voir les vrais noms de fichiers (peut-être un gros .zip ou un gros .json).
- Choisir ensuite seulement 1 ou quelques fichiers à télécharger (pour rester sous ~5 Go).


In [3]:
# Lister les fichiers du dataset Spotify Million Playlist sur Kaggle
!kaggle datasets files -d himanshuwagh/spotify-million


Next Page Token = CfDJ8JvX8PYzZ5dKharEO0H57fGkwtm7-O97SBp2k-fehVZUc1I2LW4FEVjXVJjn8v2UoYISDtRMOFeIjG4kcoBimyh3fLEPrd5g43mYvHiLOiEzQP6v7Uger9WtC9AVGQ0egWc5B0nv5SQ0VaGkQgiI5BehUYh1sr3yXQ
name                                   size  creationDate                
---------------------------------  --------  --------------------------  
README.md                              9833  2022-08-19 07:13:22.678000  
data/mpd.slice.0-999.json          34119368  2022-08-19 07:14:41.405000  
data/mpd.slice.112000-112999.json  33547633  2022-08-19 07:15:33.003000  
data/mpd.slice.113000-113999.json  33936509  2022-08-19 07:15:22.070000  
data/mpd.slice.114000-114999.json  33946205  2022-08-19 07:13:32.339000  


In [4]:
dataset_name = "himanshuwagh/spotify-million"
target_dir = "/content/mpd"

!mkdir -p $target_dir
!kaggle datasets download -d $dataset_name -f README.md -p $target_dir --unzip


Dataset URL: https://www.kaggle.com/datasets/himanshuwagh/spotify-million
License(s): unknown
  0% 0.00/9.60k [00:00<?, ?B/s]
100% 9.60k/9.60k [00:00<00:00, 14.3MB/s]


# Step 2.1 – Télécharger seulement ~4–5 Go de fichiers `mpd.slice.*.json` (2min to download)

Objectif :
- Ne PAS télécharger tout le dataset.
- Télécharger seulement un sous-ensemble de fichiers `data/mpd.slice.*.json`.
- Chaque slice fait ≈ 33–35 Mo → avec ~120 slices on obtient ≈ 4 Go.

Important :
- Dans ce dataset Kaggle, les noms de fichiers incluent le préfixe `data/`,
  par exemple : `data/mpd.slice.0-999.json`.
- Il faut donc utiliser ce nom complet dans l’option `-f` de l’API Kaggle.
- Avec `--unzip`, les fichiers seront extraits dans `/content/mpd/data/`.


In [5]:
import math
from pathlib import Path

dataset_name = "himanshuwagh/spotify-million"
target_dir = "/content/mpd"

# Nombre de slices à télécharger (ajuste si besoin)
N_SLICES = 120  # 120 slices * ~34 Mo ≈ 4 Go

# Créer le dossier cible
!mkdir -p $target_dir

print(f"Téléchargement de {N_SLICES} slices dans {target_dir} ...")

for i in range(N_SLICES):
    start = i * 1000
    end = start + 999
    kaggle_fname = f"data/mpd.slice.{start}-{end}.json"  # ⚠️ avec 'data/' devant

    print(f"\n⏬ Downloading slice {i+1}/{N_SLICES}: {kaggle_fname}")
    # -f : nom EXACT du fichier vu dans "kaggle datasets files"
    # --unzip : on extrait directement le .json dans /content/mpd/data/
    !kaggle datasets download -d $dataset_name -f $kaggle_fname -p $target_dir --unzip


Téléchargement de 120 slices dans /content/mpd ...

⏬ Downloading slice 1/120: data/mpd.slice.0-999.json
Dataset URL: https://www.kaggle.com/datasets/himanshuwagh/spotify-million
License(s): unknown
  0% 0.00/5.43M [00:00<?, ?B/s]
100% 5.43M/5.43M [00:00<00:00, 90.7MB/s]

⏬ Downloading slice 119/120: data/mpd.slice.118000-118999.json
Dataset URL: https://www.kaggle.com/datasets/himanshuwagh/spotify-million
License(s): unknown
  0% 0.00/5.54M [00:00<?, ?B/s]
100% 5.54M/5.54M [00:00<00:00, 86.6MB/s]

⏬ Downloading slice 120/120: data/mpd.slice.119000-119999.json
Dataset URL: https://www.kaggle.com/datasets/himanshuwagh/spotify-million
License(s): unknown
 55% 3.00M/5.42M [00:00<00:00, 26.1MB/s]
100% 5.42M/5.42M [00:00<00:00, 44.6MB/s]


# Step 3 – Dézipper tous les fichiers mpd.slice.*.json.zip

Objectif :
- Extraire chaque fichier `mpd.slice.*.json` du ZIP téléchargé.
- Les placer dans le dossier `/content/mpd/`.
- Vérifier ensuite combien de `.json` ont été extraits et leur taille totale.

Après cette étape :
- On doit voir des fichiers comme : `/content/mpd/mpd.slice.0-999.json`


In [6]:
import zipfile
from pathlib import Path

mpd_dir = Path("/content/mpd")

# Trouver tous les zip
zip_files = sorted(mpd_dir.glob("*.zip"))

print(f"Nombre de ZIP trouvés : {len(zip_files)}")

# Extraction
for z in zip_files:
    print(f"📦 Unzipping : {z.name}")
    with zipfile.ZipFile(z, 'r') as zip_ref:
        zip_ref.extractall(mpd_dir)

print("✔️ Extraction terminée")

# Vérifier la présence des .json
json_files = sorted(mpd_dir.glob("*.json"))
print(f"\nNombre de fichiers JSON extraits : {len(json_files)}")
print("Exemples :")
for p in json_files[:5]:
    print(" -", p.name)

# Taille totale
total_bytes = sum(p.stat().st_size for p in json_files)
total_gb = total_bytes / (1024**3)
print(f"\nTaille totale des JSON ≈ {total_gb:.2f} Go")


Nombre de ZIP trouvés : 120
📦 Unzipping : mpd.slice.0-999.json.zip
📦 Unzipping : mpd.slice.1000-1999.json.zip
📦 Unzipping : mpd.slice.10000-10999.json.zip
📦 Unzipping : mpd.slice.98000-98999.json.zip
📦 Unzipping : mpd.slice.99000-99999.json.zip
✔️ Extraction terminée

Nombre de fichiers JSON extraits : 120
Exemples :
 - mpd.slice.0-999.json
 - mpd.slice.1000-1999.json
 - mpd.slice.10000-10999.json
 - mpd.slice.100000-100999.json
 - mpd.slice.101000-101999.json

Taille totale des JSON ≈ 3.78 Go


# Step 4 – Construire un document texte riche pour chaque playlist, pour LDA

Objectif :
- Parcourir les 120 fichiers `mpd.slice.*.json` dans `/content/mpd`.
- Pour chaque playlist, créer un texte qui concatène :
  - `name` (titre de la playlist)
  - `description` (si elle existe)
  - pour chaque track :
    - `track_name`
    - `artist_name`
    - `album_name`
- Stocker tout dans un DataFrame `df_playlists` avec les colonnes :
  - `pid` : ID de la playlist
  - `playlist_name` : titre
  - `text` : document texte complet

On limite le nombre total de playlists avec `MAX_PLAYLISTS`
pour éviter les problèmes de RAM (tu peux augmenter ou diminuer).
A la fin, on sauvegarde le résultat dans un fichier Parquet :
`/content/df_playlists_text.parquet`.


In [16]:
import json
from pathlib import Path

import pandas as pd
from tqdm.auto import tqdm

mpd_dir = Path("/content/mpd")
json_files = sorted(mpd_dir.glob("mpd.slice.*.json"))

print(f"Nombre de fichiers JSON trouvés : {len(json_files)}")

MAX_PLAYLISTS = 50000  # comme avant

playlist_records = []
track_records = []      # 👈 NEW : une ligne par (playlist, track)
count_playlists = 0

for json_path in tqdm(json_files, desc="Lecture des slices"):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    playlists = data.get("playlists", [])

    for pl in playlists:
        pid = pl.get("pid")
        pl_name = (pl.get("name") or "").strip()
        pl_desc = (pl.get("description") or "").strip()

        track_tokens = []
        for tr in pl.get("tracks", []):
            track_uri   = (tr.get("track_uri")   or "").strip()
            track_name  = (tr.get("track_name")  or "").strip()
            artist_name = (tr.get("artist_name") or "").strip()
            album_name  = (tr.get("album_name")  or "").strip()

            # texte pour LDA
            track_tokens.extend([track_name, artist_name, album_name])

            # 👇 NEW : on garde une ligne par track
            track_records.append({
                "pid": pid,
                "track_uri": track_uri,
                "track_name": track_name,
                "artist_name": artist_name,
                "album_name": album_name
            })

        full_text_parts = [
            pl_name,
            pl_desc,
            " ".join(track_tokens)
        ]
        full_text = " ".join(part for part in full_text_parts if part)

        playlist_records.append(
            {
                "pid": pid,
                "playlist_name": pl_name,
                "text": full_text
            }
        )

        count_playlists += 1
        if count_playlists >= MAX_PLAYLISTS:
            break

    if count_playlists >= MAX_PLAYLISTS:
        break

print(f"\nNombre total de playlists chargées : {len(playlist_records)}")
print(f"Nombre total de (playlist, track) lignes : {len(track_records)}")

# DataFrame playlists
df_playlists = pd.DataFrame(playlist_records)
print(df_playlists.head())
print(df_playlists.shape)

# 👇 NEW : DataFrame tracks
df_pl_tracks = pd.DataFrame(track_records)
print(df_pl_tracks.head())
print(df_pl_tracks.shape)

# Sauvegarde
df_playlists.to_parquet("/content/df_playlists_text.parquet", index=False)
df_pl_tracks.to_parquet("/content/df_pl_tracks.parquet", index=False)

print("\n✅ df_playlists_text.parquet et df_pl_tracks.parquet sauvegardés")


Nombre de fichiers JSON trouvés : 120


Lecture des slices:   0%|          | 0/120 [00:00<?, ?it/s]


Nombre total de playlists chargées : 50000
Nombre total de (playlist, track) lignes : 3353026
   pid     playlist_name                                               text
0    0        Throwbacks  Throwbacks Lose Control (feat. Ciara & Fat Man...
1    1  Awesome Playlist  Awesome Playlist Eye of the Tiger Survivor Eye...
2    2            korean  korean Like You Hoody On And On GOOD (feat. EL...
3    3               mat  mat Danse macabre Camille Saint-Saëns French F...
4    4               90s  90s Tonight, Tonight The Smashing Pumpkins Mel...
(50000, 3)
   pid                             track_uri  \
0    0  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
1    0  spotify:track:6I9VzXrHxO9rA9A5euc8Ak   
2    0  spotify:track:0WqIKmW4BTrj3eJFmnCKMv   
3    0  spotify:track:1AWQoqb9bSvzTjaLralEkT   
4    0  spotify:track:1lzr43nnXAijIGYnCT8M8H   

                                   track_name        artist_name  \
0  Lose Control (feat. Ciara & Fat Man Scoop)      Missy Elliott   
1            

# Step 5 — Nettoyage du texte + Vectorisation (Bag of Words)

Objectif :
- Recharger le DataFrame avec les 50k playlists.
- Nettoyer légèrement le texte (`lowercase`, retirer caractères non-alphanumériques).
- Construire une matrice sparse (documents × mots) avec `CountVectorizer`.
- Configuration de CountVectorizer :
  - `max_features=50000` : limite la taille du vocabulaire → RAM maîtrisée
  - `min_df=10` : garde seulement les mots présents dans ≥ 10 playlists
  - `max_df=0.5` : enlève les mots trop fréquents (présents dans >50% des playlists)

Résultat :
- Une matrice sparse `X_bow` qui servira d'entrée au modèle LDA.


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re

# (Re)charger le DataFrame sauvegardé
df_playlists = pd.read_parquet("/content/df_playlists_text.parquet")

print("Shape du DataFrame :", df_playlists.shape)

# 🔹🔹🔹 NEW : on garde 1 "slice" pour le test 🔹🔹🔹
# Ici : on prend les 1000 premières playlists comme set de test,
# et le reste sert à entraîner LDA (train).
TEST_SLICE_SIZE = 1000  # 1 "slice" ~ 1000 playlists

df_playlists_test = df_playlists.iloc[:TEST_SLICE_SIZE].copy()
df_playlists      = df_playlists.iloc[TEST_SLICE_SIZE:].copy()  # df_playlists = TRAIN


print("Shape TRAIN :", df_playlists.shape)
print("Shape TEST  :", df_playlists_test.shape)
# 🔹🔹🔹 FIN DU CHANGEMENT 🔹🔹🔹

# 1) Petit nettoyage du texte
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-z0-9]+", " ", text)
    return text

df_playlists["clean_text"] = df_playlists["text"].astype(str).apply(clean_text)

# 2) Vectorisation (Bag of Words)
vectorizer = CountVectorizer(
    max_features=50000,  # limite RAM
    min_df=10,           # mot doit être dans au moins 10 playlists
    max_df=0.5           # élimine les mots trop fréquents
)

print("⏳ Construction de la matrice BoW ...")
X_bow = vectorizer.fit_transform(df_playlists["clean_text"])
print("✔️ Matrice BoW créée")
print("Shape :", X_bow.shape)


Shape du DataFrame : (50000, 3)
Shape TRAIN : (49000, 3)
Shape TEST  : (1000, 3)
⏳ Construction de la matrice BoW ...
✔️ Matrice BoW créée
Shape : (49000, 34320)


# Step 6 V2 — Entraîner un modèle LDA sur les playlists (12' do train)
(**UPDATE** : L’erreur dit : OSError: [Errno 28] No space left on device → joblib essaie de créer des fichiers temporaires pour le parallélisme (n_jobs=-1), et ça sature le /tmp de Colab.)

Objectif :
- Appliquer un modèle LDA (Latent Dirichlet Allocation) sur la matrice BoW `X_bow`.
- Obtenir pour chaque playlist un vecteur de dimension `N_TOPICS` qui représente
  sa distribution de "topics" (thèmes musicaux).
- Ces vecteurs serviront ensuite à calculer la similarité entre playlists.

Paramètres :
- `N_TOPICS = 50` (tu peux essayer 30, 50, 80…)
- `learning_method="online"` : plus efficace sur des grandes matrices.
- `max_iter=10` : nombre d'itérations d'apprentissage.




In [10]:
from sklearn.decomposition import LatentDirichletAllocation

N_TOPICS = 30  # un peu plus léger que 50

lda = LatentDirichletAllocation(
    n_components=N_TOPICS,
    learning_method="online",
    max_iter=10,
    random_state=42,
    n_jobs=1   # ❗ très important : plus de parallélisation → plus de tmp sur disque
)

print("⏳ Entraînement du modèle LDA (n_jobs=1) ...")
playlist_topic_matrix = lda.fit_transform(X_bow)
print("✔️ LDA entraîné")

print("Shape de la matrice topics (playlists × topics) :", playlist_topic_matrix.shape)



⏳ Entraînement du modèle LDA (n_jobs=1) ...
✔️ LDA entraîné
Shape de la matrice topics (playlists × topics) : (49000, 30)


# Step 6.1 — Donne les thèmes et le mots clefs => les dimensions

In [11]:
def print_top_words(model, feature_names, n_top_words=15):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\n🟦 Topic #{topic_idx}")
        top_indices = topic.argsort()[::-1][:n_top_words]
        top_words = [feature_names[i] for i in top_indices]
        print("  " + ", ".join(top_words))

feature_names = vectorizer.get_feature_names_out()
print_top_words(lda, feature_names, n_top_words=15)



🟦 Topic #0
  eminem, lil, fetty, wap, wayne, khalifa, wiz, rich, drake, money, remix, young, big, dj, kid

🟦 Topic #1
  vol, album, goes, pop, punk, punch, bts, five, pt, moment, beautiful, red, exo, mini, wings

🟦 Topic #2
  remix, ep, flume, remixes, odesza, wonder, summer, down, so, high, oh, go, feel, light, night

🟦 Topic #3
  original, soundtrack, motion, picture, cast, broadway, recording, hamilton, williams, john, harry, music, score, zimmer, hans

🟦 Topic #27
  justin, bieber, chainsmokers, know, selena, do, gomez, sheeran, ed, shawn, ariana, let, grande, edition, mendes

🟦 Topic #28
  john, acoustic, live, james, mayer, heart, ed, sheeran, jack, at, head, lumineers, home, ben, ep

🟦 Topic #29
  drake, kanye, west, cole, weeknd, kendrick, lamar, kid, late, rapper, views, childish, gambino, hills, chance


# Step 7 — Calculer la similarité entre playlists (cosine similarity)

Objectif :
- Utiliser la matrice `playlist_topic_matrix` (50000 × 30).
- Pour une playlist donnée, calculer la similarité cosinus avec toutes les autres.
- Trier les résultats pour obtenir les playlists les plus similaires.
- Retourner pour chaque playlist choisie :
  - `pid`
  - `playlist_name`
  - `similarity`

Note :
- On ne pré-calcule PAS la matrice 50000×50000 (trop lourde).
- On calcule la similarité **uniquement pour une playlist à la demande**.


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_similar_playlists(idx, top_k=10):
    """
    idx : index dans df_playlists de la playlist de référence
    top_k : nombre de playlists similaires à retourner
    """

    query_vec = playlist_topic_matrix[idx].reshape(1, -1)
    sims = cosine_similarity(query_vec, playlist_topic_matrix)[0]

    # Trier : du plus similaire au moins similaire
    sorted_idx = np.argsort(-sims)

    # On ignore la playlist elle-même
    sorted_idx = [i for i in sorted_idx if i != idx]

    top_idx = sorted_idx[:top_k]

    result = df_playlists.iloc[top_idx][["pid", "playlist_name"]].copy()
    result["similarity"] = sims[top_idx]

    return result

# Exemple : playlist index 0
example_idx = 0
print("Playlist de référence :", df_playlists.iloc[example_idx]["playlist_name"])
similar = get_similar_playlists(example_idx, top_k=5)
similar


Playlist de référence : disney


,pid,playlist_name,similarity
43019,3019,Hindi songs,0.988460
10534,107534,Showtunes,0.988079
16421,112421,Kiddos,0.984878
33037,20037,Once Upon A Time,0.984846
16079,112079,Musicals,0.984197


Le tableau obtenu montre, pour une playlist de référence (“Throwbacks”), les playlists les plus similaires identifiées par notre modèle.
- La similarité est calculée en comparant les vecteurs thématiques LDA de chaque playlist : lorsque deux playlists contiennent des morceaux, artistes et albums proches, leurs distributions de topics deviennent quasi identiques, ce qui produit une similarité cosinus très proche de 1.
- Ainsi, les playlists “Middle School”, “TBT”, “Yonce” ou encore une autre “Throwbacks” apparaissent avec des scores supérieurs à 0.99, ce qui indique qu’elles appartiennent au même univers musical (nostalgie, R&B/pop des années 2000, oldies).
- En bref, un score proche de 1 signifie que deux playlists partagent pratiquement les mêmes contenus musicaux et donc la même thématique.

# Step 9 — Sauvegarder tout le modèle dans un dossier unique

Objectif :
- Créer un dossier `playlist_model/`
- Sauvegarder dedans :
  - le modèle LDA
  - le CountVectorizer
  - la matrice topics des playlists
  - le DataFrame (pid, nom, texte)
- Zipper le dossier pour pouvoir le télécharger facilement

Ce dossier contiendra TOUT pour recharger le système
sans devoir ré-entraîner LDA ni recalculer BoW.


In [17]:
import joblib
import pandas as pd
import os
import shutil

# 1) Créer dossier propre
MODEL_DIR = "/content/playlist_model"
os.makedirs(MODEL_DIR, exist_ok=True)

# 2) Sauvegarder les objets sklearn
joblib.dump(lda, os.path.join(MODEL_DIR, "lda_model.joblib"))
joblib.dump(vectorizer, os.path.join(MODEL_DIR, "vectorizer.joblib"))
joblib.dump(playlist_topic_matrix, os.path.join(MODEL_DIR, "playlist_topic_matrix.joblib"))

# 3) Sauvegarder les DataFrames nécessaires
#    ⚠️ Ici on suppose que df_playlists = TRAIN
#    et que df_playlists_test existe déjà (ta slice mise de côté)
df_playlists.to_parquet(os.path.join(MODEL_DIR, "df_playlists.parquet"), index=False)
df_pl_tracks.to_parquet(os.path.join(MODEL_DIR, "df_pl_tracks.parquet"), index=False)
df_playlists_test.to_parquet(os.path.join(MODEL_DIR, "test_slices.parquet"), index=False)

print("✔️ df_playlists.parquet (TRAIN) sauvegardé")
print("✔️ df_pl_tracks.parquet sauvegardé")
print("✔️ test_slices.parquet (TEST) sauvegardé")

# 4) Créer un ZIP téléchargeable
ZIP_PATH = "/content/playlist_model.zip"
shutil.make_archive("/content/playlist_model", 'zip', MODEL_DIR)

print("📦 ZIP créé :", ZIP_PATH)


✔️ df_playlists.parquet (TRAIN) sauvegardé
✔️ df_pl_tracks.parquet sauvegardé
✔️ test_slices.parquet (TEST) sauvegardé
📦 ZIP créé : /content/playlist_model.zip


# Step 10 — Recharger le modèle complet depuis le dossier playlist_model
Après cette étape, tu peux directement appeler get_similar_playlists().


In [ ]:
import joblib
import pandas as pd

MODEL_DIR = "/content/playlist_model"  # ou le dossier dans ton Drive

lda = joblib.load(f"{MODEL_DIR}/lda_model.joblib")
vectorizer = joblib.load(f"{MODEL_DIR}/vectorizer.joblib")
playlist_topic_matrix = joblib.load(f"{MODEL_DIR}/playlist_topic_matrix.joblib")
df_playlists = pd.read_parquet(f"{MODEL_DIR}/df_playlists.parquet")

print("✔️ Modèle complètement rechargé")
